## Feature Engineering

In [23]:
import pandas as pd
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
import numpy as np
from gensim.models import FastText
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sentence_transformers import SentenceTransformer

In [19]:
#read tracks_spotify_lang - 
#the file contains the detected language created with the get_language script
tracks_lang=pd.read_csv('data/tracks_spotify_lang.csv')
#Remove rows of repeating lyrics
tracks_lang_no_dup = tracks_lang.drop_duplicates(subset='lyrics', keep='first').reset_index()

#get counts of languages
tracks_lang_no_dup['language'].value_counts().head(10)

#filter for only the english ones
filtered_df = tracks_lang_no_dup[tracks_lang_no_dup['language'].isin(['en'])]

#lowercasing the lyrics and saving them in new column
filtered_df['lyrics_clean'] = filtered_df['lyrics'].str.lower()
#removing digits
filtered_df['lyrics_clean'] = filtered_df['lyrics_clean'].apply(lambda x: re.sub(r'\d+', '', x))
#removing punctuation
filtered_df['lyrics_clean'] = filtered_df['lyrics_clean'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
#extend stopwords
new_stop_words = ['aba','ooh','yeah','hey','whoa','woah', 'ohh', 'was', 'mmm', 'oooh','yah','yeh', 'hmm','deh','doh','jah','wa','la','na','oh','em','da','dem','mi','di','fi','gyal','dey','ey','eh','inna','nuh','pon','ba','ye','de','weh','gal','oo','bam','wid','mek','seh','aah','li','ni','waan','wey']

#define a function that removes english stopwords
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    stop_words.update(new_stop_words) 
    words = word_tokenize(text)
    return " ".join([word for word in words if word.lower() not in stop_words])

#define a funtion that tokenizes the lyrics 
def tokenize_text(text):
    return nltk.word_tokenize(str(text))

#apply the defined funtions
filtered_df['lyrics_clean'] = filtered_df['lyrics_clean'].apply(remove_stopwords)
filtered_df['lyrics_clean'] = filtered_df['lyrics_clean'].apply(tokenize_text)

#lemmatizing lyrics
lemmatizer = WordNetLemmatizer()
filtered_df['lyrics_clean'] = filtered_df['lyrics_clean'].apply(lambda words: [lemmatizer.lemmatize(word) for word in words])

#Join list items into a single string without commas 'untokenize'
filtered_df['lyrics_clean_untok'] = filtered_df['lyrics_clean'].apply(' '.join)

#eliminate rows that contain characters outside the usual English alphabet
pattern = '^[a-zA-Z\s,.;?!-]*$'
filtered_df = filtered_df.dropna(subset=['lyrics_clean_untok'])
filtered_df = filtered_df[filtered_df['lyrics_clean_untok'].str.contains(pattern)]

#filter the data based on duration and word count
filtered_df= filtered_df[(filtered_df['duration_s'] >= 90) & (filtered_df['duration_s'] <= 600)]
filtered_df= filtered_df_short[(filtered_df_short['word_count'] >= 500) & (filtered_df_short['word_count'] <= 6000)]

filtered_df['lyrics_str'] = filtered_df['lyrics'].astype(str)

/var/folders/60/s86dq17j5zd4vn2hf6ywd3200000gn/T/ipykernel_46582/1371305762.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['lyrics_clean'] = filtered_df['lyrics'].str.lower()
/var/folders/60/s86dq17j5zd4vn2hf6ywd3200000gn/T/ipykernel_46582/1371305762.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['lyrics_clean'] = filtered_df['lyrics_clean'].apply(lambda x: re.sub(r'\d+', '', x))
/var/folders/60/s86dq17j5zd4vn2hf6ywd3200000gn/T/ipykernel_46582/1371305762.py:18: SettingWit

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/tomtom_ghost/opt/anaconda3/envs/py310/lib/python3.10/site-packages/pandas/core/indexes/base.py", line 3803, in get_loc
    raise
  File "pandas/_libs/index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 165, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 5745, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 5753, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'duration_s'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/tomtom_ghost/opt/anaconda3/envs/py310/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/60/s86dq17j5zd4vn2hf6ywd3200000gn/T/ipykernel_46582/1371305762.py", line 50, in

### TF-IDF

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
df_tfidf = vectorizer.fit_transform(filtered_df['lyrics_str'])
df_tfidf

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/tomtom_ghost/opt/anaconda3/envs/py310/lib/python3.10/site-packages/pandas/core/indexes/base.py", line 3803, in get_loc
    raise
  File "pandas/_libs/index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 165, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 5745, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 5753, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'lyrics_str'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/tomtom_ghost/opt/anaconda3/envs/py310/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/60/s86dq17j5zd4vn2hf6ywd3200000gn/T/ipykernel_46582/55503419.py", line 4, in <m

### Word2Vec

In [22]:
model_w2v = Word2Vec(sentences=filtered_df['lyrics_clean'].tolist(), vector_size=100, window=5, min_count=1, workers=4)

def get_sentence_vector(words):
    word_vectors = [model_w2v.wv[word] for word in words if word in model_w2v.wv.index_to_key]
    if not word_vectors:
        return np.zeros(model_w2v.vector_size)
    return np.mean(word_vectors, axis=0)

filtered_df['w2v_vector'] = filtered_df['lyrics_clean'].apply(get_sentence_vector)


### GloVe

In [20]:
# You'll need to convert the GloVe file format to word2vec format
# You can download pre-trained GloVe embeddings from the GloVe website
glove_input_file = 'glove.6B.100d.txt'  # Modify the path as per your downloaded model
word2vec_output_file = 'glove.6B.100d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)

# Load the model
glove_model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

def get_glove_sentence_vector(words):
    word_vectors = [glove_model[word] for word in words if word in glove_model.key_to_index]
    if not word_vectors:
        return np.zeros(glove_model.vector_size)
    return np.mean(word_vectors, axis=0)

filtered_df['glove_vector'] = filtered_df['lyrics_clean'].apply(get_glove_sentence_vector)


/var/folders/bf/3n7wxjyn0rl9rrhzmyw8mb7w0000gn/T/ipykernel_19875/3474118581.py:9: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_input_file, word2vec_output_file)
/var/folders/bf/3n7wxjyn0rl9rrhzmyw8mb7w0000gn/T/ipykernel_19875/3474118581.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['glove_vector'] = filtered_df['lyrics_clean'].apply(get_glove_sentence_vector)


### FastText

In [25]:
model_ft = FastText(vector_size=100, window=5, min_count=1, sentences=filtered_df['lyrics_clean'].tolist(), epochs=10)

def get_ft_sentence_vector(words):
    word_vectors = [model_ft.wv[word] for word in words if word in model_ft.wv.index_to_key]
    if not word_vectors:
        return np.zeros(model_ft.vector_size)
    return np.mean(word_vectors, axis=0)

filtered_df['ft_vector'] = filtered_df['lyrics_clean'].apply(get_ft_sentence_vector)


/var/folders/bf/3n7wxjyn0rl9rrhzmyw8mb7w0000gn/T/ipykernel_19875/3247512462.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['ft_vector'] = filtered_df['lyrics_clean'].apply(get_ft_sentence_vector)


### Doc2Vec

In [22]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(filtered_df['lyrics_clean'].tolist())]
model_d2v = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4, epochs=100)
model_d2v.build_vocab(documents)
model_d2v.train(documents, total_examples=model_d2v.corpus_count, epochs=model_d2v.epochs)

filtered_df['d2v_vector'] = [model_d2v.dv[i] for i in range(len(filtered_df['lyrics_clean']))]


/var/folders/bf/3n7wxjyn0rl9rrhzmyw8mb7w0000gn/T/ipykernel_19875/794095595.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['d2v_vector'] = [model_d2v.dv[i] for i in range(len(filtered_df['lyrics_clean']))]


### Sentence Embeddings (sBert)


In [26]:
# Initialize the model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Generate embeddings
filtered_df['sbert_vector'] = filtered_df['lyrics_clean'].apply(lambda x: model.encode(' '.join(x)))


/var/folders/bf/3n7wxjyn0rl9rrhzmyw8mb7w0000gn/T/ipykernel_19875/2840678984.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['sbert_vector'] = filtered_df['lyrics_clean'].apply(lambda x: model.encode(' '.join(x)))


In [27]:
filtered_df

,index,acousticness,album_id,analysis_url,artists_id,available_markets,country,danceability,disc_number,duration_ms,...,type,language,confidence,lyrics_clean,lyrics_str,w2v_vector,glove_vector,d2v_vector,ft_vector,sbert_vector
0,0,0.29400,0D3QufeCudpQANOR7luqdr,https://api.spotify.com/v1/audio-analysis/5qlj...,['3mxJuHRn2ZWD5OofvJtDZY'],"['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH...",BE,0.698,1.0,235584.0,...,track,en,0.999995,"[perhaps, bound, restless, always, yearning, n...",perhaps bound restless always yearning never s...,"[0.6544756, -0.3229556, -0.139516, 0.22122717,...","[0.016300496, 0.34271294, 0.37241003, -0.16385...","[-1.5367717, -2.4504938, 0.15364909, -3.299925...","[1.0889084, -0.17906623, -1.5707855, -1.269119...","[-0.046746217, -0.17634994, 0.40579668, -0.019..."
1,1,0.86300,1bcqsH5UyTBzmh9YizdsBE,https://api.spotify.com/v1/audio-analysis/3VAX...,['4xWMewm6CYMstu0sPgd9jJ'],"['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH...",BE,0.719,1.0,656960.0,...,track,en,0.999997,"[gods, godsdo, know, stronger, native, proverb...",gods godsdo know stronger native proverb east ...,"[0.22900137, -0.011180511, 0.11696933, -0.0378...","[-0.012346554, 0.142385, 0.20282121, -0.249358...","[2.0670466, -1.9952976, -0.78856546, -1.299497...","[0.6557791, 0.046614774, -1.3376333, -0.805199...","[0.18349041, 0.17910825, -0.0504529, -0.278700..."
2,2,0.75000,4tKijjmxGClg4JOLAyo2qE,https://api.spotify.com/v1/audio-analysis/1L3Y...,['3hYaK5FF3YAglCj5HZgBnP'],['GB'],BE,0.466,1.0,492840.0,...,track,en,0.999995,"[closed, love, didnt, need, pain, twice, enoug...",closed love didnt need pain twice enough vain ...,"[0.6947316, -0.28599197, -0.29491928, 0.682063...","[-0.07529787, 0.18532664, 0.25859946, -0.17300...","[-1.1369686, 0.56759566, -3.5360346, -0.316745...","[0.71569294, -0.3852827, -1.5801319, 0.3134563...","[-0.27127653, 0.33813798, 0.26504812, 0.256099..."
3,3,0.76300,6FeJF5r8roonnKraJxr4oB,https://api.spotify.com/v1/audio-analysis/6aCe...,['2KQsUB9DRBcJk17JWX1eXD'],"['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH...",BE,0.719,1.0,316578.0,...,track,en,0.999998,"[pain, n, uncomfortable, frame, mind, may, phy...",pain n uncomfortable frame mind may physical b...,"[-0.03170229, -0.06108123, 0.18263377, -0.0254...","[-0.07350073, 0.19990432, 0.21373656, 0.010531...","[-1.0665642, 2.9360483, 2.1036134, -2.8190637,...","[0.33878967, -0.22557345, -0.9858403, -0.97439...","[0.05471897, 0.29905123, 0.18562856, -0.030469..."
4,5,0.97100,7CCwkPweMxKq8yWkVerH6T,https://api.spotify.com/v1/audio-analysis/4PrA...,['3kzwYV3OCB010YfXMF0Avt'],"['AE', 'AR', 'BH', 'BO', 'BR', 'CL', 'CO', 'CR...",BE,0.367,1.0,183653.0,...,track,en,0.999997,"[ocean, wed, wading, distance, would, water, c...",ocean wed wading distance would water creating...,"[0.23468369, 0.123190805, 0.14694338, 0.348222...","[-0.2001696, 0.29664358, 0.30343485, -0.061161...","[-2.7153978, 0.94972116, 0.113015994, 0.052836...","[0.7678717, -0.062007383, -0.8720988, -1.32967...","[0.07439018, 0.18220776, 0.2588866, 0.04634476..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40811,101933,0.00837,4PXy3cBCNeY0ZVKTOGi9Cw,https://api.spotify.com/v1/audio-analysis/0ujk...,['3yW6jTzGjHUUkLvLkjLOVn'],"['AU', 'NZ']",AR,0.553,1.0,251253.0,...,track,en,0.999997,"[black, fingernails, red, wine, wan, na, make,...",black fingernails red wine wan na make mine lo...,"[0.44811702, -0.100758, 0.2660182, 0.06938852,...","[-0.048288547, 0.2610699, 0.3593745, -0.176603...","[1.925146, -2.844099, 1.6665615, -1.1851282, -...","[0.5904623, 0.23795933, -1.4836384, -0.4944968...","[0.14479434, 0.21599376, 0.22885923, 0.2102713..."
40812,101934,0.00564,1M9n4vCmOH4lbcHrpt21Qy,https://api.spotify.com/v1/audio-analysis/4e5w...,['6n3YUZcayLRuAunJUUelvz'],"['AU', 'NZ']",AR,0.602,1.0,178893.0,...,track,en,0.999998,"[got, killed, black, bugs, video, game, go, do...",got killed black bugs video game go doesnt mea...,"[0.82594174, 0.50977314, -0.18910752, 0.247966...","[0.